# Vascular Scan Analysis Demo

This notebook walks through the tiled vascular analysis pipeline end-to-end.
You can: (1) configure per-stack settings, (2) run the new global + tiled
processing path, and (3) visualize intensities, vesselness, binaries, and traced paths.

## How to use this notebook

1. Point `STACK_PATH` at the 3D image stack you want to analyze.
2. Adjust `demo_overrides` to experiment with tiling/block parameters.
3. Run the pipeline cell to generate results.
4. Use the visualization/QC sections to inspect what happened.

In [ ]:
%matplotlib inline
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path('..').resolve()
SRC_ROOT = PROJECT_ROOT / 'src'
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)

## Input locations
Update the `STACK_PATH` below to point at the dataset you wish to process.

In [ ]:
from datetime import datetime

DATA_ROOT = Path('../test/test_files').resolve()
STACK_PATH = DATA_ROOT / '240207_002.czi'  # TODO: update to your dataset
CONFIG_PATH = Path('../config/default_vessel_config.yaml').resolve()
OUTPUT_DIR = (PROJECT_ROOT / 'outputs' / 'vascular_scan_demo').resolve()
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f'Project root : {PROJECT_ROOT}')
print(f'Data root    : {DATA_ROOT}')
print(f'Stack path   : {STACK_PATH}')
print(f'Config path  : {CONFIG_PATH}')
print(f'Output dir   : {OUTPUT_DIR}')

if not STACK_PATH.exists():
    raise FileNotFoundError('Update STACK_PATH to point at an existing stack before continuing.')

## Configure the stack-specific pipeline
This block loads the YAML defaults, applies demo overrides, and prints the
fingerprint so you can keep outputs reproducible.

In [ ]:
from dataclasses import asdict
from VesselTracer.config import (
    load_stack_config,
    apply_stack_overrides,
    config_fingerprint,
    save_stack_config,
)

default_stack_cfg = load_stack_config(CONFIG_PATH)

demo_overrides = {
    'remove_dead_frames': True,
    'dead_frame_threshold': 2.5,
    'background_mode': 'polyfit',
    'smoothing': 'gaussian',
    'notes': {'debug_level': 2, 'purpose': 'VascularScanAnalysis demo'},
    'tiling': {
        'use_tiled_pipeline': True,
        'tile_xy': 320,
        'tile_overlap': 80,
        'halo_xy': 24,
        'vessel_scales': (1.0, 1.6, 2.3, 3.2, 4.5),
        'thresh_mode': 'sauvola',
        'blend_mode': 'vesselness_max',
        'cache_intermediates': False,
    },
}

stack_cfg = apply_stack_overrides(default_stack_cfg, demo_overrides)
fingerprint = config_fingerprint(stack_cfg)
print(f'Active stack fingerprint: {fingerprint}')

stack_df = pd.Series(asdict(stack_cfg))
stack_df

## Instantiate the controller

In [ ]:
from VesselTracer.controller import VesselAnalysisController

os.environ.setdefault('VA_DEBUG', str(stack_cfg.notes.get('debug_level', 1)))

controller = VesselAnalysisController(
    input_path=STACK_PATH,
    config_path=CONFIG_PATH,
    verbose=stack_cfg.notes.get('debug_level', 1),
    use_gpu=False,
    stack_config=stack_cfg,
)

print(f'Controller ready (fingerprint={controller.stack_fingerprint[:8]})')

## Run the global + tiled pipeline

In [ ]:
%%time
controller.run_analysis(stack_config=stack_cfg)
roi = controller.roi_model

## Summaries

In [ ]:
def summarize_volume(name, volume):
    if volume is None:
        return None
    arr = np.asarray(volume)
    return {
        'name': name,
        'shape': arr.shape,
        'dtype': arr.dtype,
        'min': float(arr.min()),
        'max': float(arr.max()),
        'mean': float(arr.mean()),
    }

summary_rows = []
summary_rows.append(summarize_volume('Processed ROI', roi.volume))
summary_rows.append(summarize_volume('Background', getattr(roi, 'background', None)))
summary_rows.append(summarize_volume('Vesselness', getattr(roi, 'vesselness', None)))
summary_rows.append(summarize_volume('Radius map', getattr(roi, 'radius', None)))
summary_rows.append(summarize_volume('Binary', getattr(roi, 'binary', None)))
summary_rows = [row for row in summary_rows if row]
pd.DataFrame(summary_rows)

## Visualization helpers

In [ ]:
import matplotlib.gridspec as gridspec

def _pick_z_indices(z_len, num_slices):
    return np.linspace(0, z_len - 1, num=min(num_slices, z_len), dtype=int)

def show_slice_grid(volume, title, num_slices=6, cmap='magma'):
    if volume is None:
        print(f'{title}: volume missing')
        return
    vol = np.asarray(volume)
    z_indices = _pick_z_indices(vol.shape[0], num_slices)
    fig, axes = plt.subplots(1, len(z_indices), figsize=(3 * len(z_indices), 4))
    for ax, z in zip(axes, z_indices):
        ax.imshow(vol[z], cmap=cmap)
        ax.set_title(f'z={z}')
        ax.axis('off')
    fig.suptitle(title)
    plt.show()

def show_mips(volume, title, cmap='inferno'):
    if volume is None:
        print(f'{title}: volume missing')
        return
    vol = np.asarray(volume)
    mip_xy = vol.max(axis=0)
    mip_xz = vol.max(axis=1)
    mip_yz = vol.max(axis=2)
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    axes[0].imshow(mip_xy, cmap=cmap)
    axes[0].set_title('XY MIP')
    axes[1].imshow(mip_xz, cmap=cmap)
    axes[1].set_title('XZ MIP')
    axes[2].imshow(mip_yz, cmap=cmap)
    for ax in axes:
        ax.axis('off')
    fig.suptitle(title)
    plt.show()

def plot_binary_overlay(vesselness, binary, title):
    if vesselness is None or binary is None:
        print('Need both vesselness and binary volumes')
        return
    v = np.asarray(vesselness)
    b = np.asarray(binary)
    mip_v = v.max(axis=0)
    mip_b = b.max(axis=0)
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(mip_v, cmap='plasma', alpha=0.7)
    ax.imshow(mip_b, cmap='gray', alpha=0.5)
    ax.set_title(title)
    ax.axis('off')
    plt.show()

def plot_paths_overlay(binary, paths, max_paths=50):
    if binary is None or not paths:
        print('Binary volume or paths missing')
        return
    mip = np.asarray(binary).max(axis=0)
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(mip, cmap='gray')
    for idx, path in enumerate(paths.values()):
        if idx >= max_paths:
            break
        coords = np.asarray(path.get('coordinates'))
        if coords.ndim != 2:
            continue
        ax.plot(coords[:, 2], coords[:, 1], linewidth=0.6)
    ax.set_title(f'Path overlays (showing up to {max_paths})')
    ax.axis('off')
    plt.show()

## Inspect intensities

In [ ]:
raw_roi = controller.image_model.create_roi(controller.config)
show_slice_grid(raw_roi.volume, 'Raw ROI slices')
show_slice_grid(roi.volume, 'Processed ROI slices', cmap='magma')
show_mips(roi.volume, 'Processed ROI MIPs')

## Vesselness vs binary

In [ ]:
show_slice_grid(getattr(roi, 'vesselness', None), 'Vesselness slices', cmap='plasma')
plot_binary_overlay(getattr(roi, 'vesselness', None), getattr(roi, 'binary', None), 'Vesselness (color) + Binary (gray)')

## Path overlays

In [ ]:
plot_paths_overlay(getattr(roi, 'binary', None), getattr(roi, 'paths', {}), max_paths=75)

## Analysis DataFrames

In [ ]:
analysis_dfs = controller.generate_analysis_dataframes()
        for name, df in analysis_dfs.items():
            print(f'
{name} ({len(df)} rows)')
            display(df.head())

## Persist configuration fingerprint

In [ ]:
cfg_path = OUTPUT_DIR / f'stack_cfg_{controller.stack_fingerprint}.json'
save_stack_config(stack_cfg, cfg_path)
print(f'Saved stack configuration to {cfg_path}')